<font color = green >

# <center>Homework<center>
</font>

__Exploratory Data Analysis (EDA)__ of car insurance dataset.
    
__Dataset description:__ Car insurance claim. As you are working in the insurance company. Company wants to know the reason why claim was not made. Doing so would allow insurance company to improve there policy for giving loan to the customer. In this project you are dealing with various feature such as age, occupation etc. based on that let's get back to the final conculsion.    

__Steps:__
- Data Cleaning
- Descriptive Statistics
- Data Visualization
- Conclusions


### <center>The dataset<center>

| Column | Description                                                                                                                                   |
|--------|-----------------------------------------------------------------------------------------------------------------------------------------------|
| `id` | unique client identifier                                                                                                                      |
| `age` | client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul>                                        |
| `gender` | client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul>                                                                         |
| `driving_experience` | the number of years a client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul>         |
| `education` | client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul>                        |
| `income` | client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul>      |
| `credit_score` | client's credit score (between zero and one)                                                                                                  |
| `vehicle_ownership` | client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul>                                               |
| `married` | client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul>                                                          |
| `children` | client's number of children                                                                                                                   |
| `postal_code` | client's postal code                                                                                                                          | 
| `annual_mileage` | number of miles driven by the client each year                                                                                                |
| `vehicle_type` | type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul>                                                                        |
| `speeding_violations` | total number of speeding violations received by the client                                                                                    | 
| `duis` | number of times the client has been caught driving under the influence of of alcohol, drugs, or other substances                              |
| `past_accidents` | total number of previous accidents the client has been involved in                                                                            |
| `outcome` | whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul>     |

In [4]:
import pandas as pd
claims = pd.read_csv("./datasets/car_insurance_claim.csv")
claims.head()

,Unnamed: 0,ID,AGE,GENDER,RACE,DRIVING_EXPERIENCE,EDUCATION,INCOME,CREDIT_SCORE,VEHICLE_OWNERSHIP,VEHICLE_YEAR,MARRIED,CHILDREN,POSTAL_CODE,ANNUAL_MILEAGE,VEHICLE_TYPE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,OUTCOME
0,0,569520,65+,female,majority,0-9y,high school,upper class,0.629027313918201,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,1,750365,16-25,male,majority,0-9y,none,poverty,0.3577571170184627,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,2,199901,16-25,female,majority,0-9y,high school,working class,0.4931457852181982,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,3,478866,16-25,male,majority,0-9y,university,working class,0.2060128507324556,1.0,before 2015,0.0,none,32765,11000.0,sedan,0,0,0,0.0
4,4,731664,26-39,male,majority,10-19y,none,working class,0.3883658881572184,1.0,before 2015,0.0,0.0,32765.0,12000.0,sedan,2,0,1,1.0


In [5]:
claims.columns= claims.columns.str.lower()

In [6]:
claims.head()

,unnamed: 0,id,age,gender,race,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,0,569520,65+,female,majority,0-9y,high school,upper class,0.629027313918201,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,1,750365,16-25,male,majority,0-9y,none,poverty,0.3577571170184627,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,2,199901,16-25,female,majority,0-9y,high school,working class,0.4931457852181982,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,3,478866,16-25,male,majority,0-9y,university,working class,0.2060128507324556,1.0,before 2015,0.0,none,32765,11000.0,sedan,0,0,0,0.0
4,4,731664,26-39,male,majority,10-19y,none,working class,0.3883658881572184,1.0,before 2015,0.0,0.0,32765.0,12000.0,sedan,2,0,1,1.0


In [7]:
claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   unnamed: 0           10000 non-null  int64 
 1   id                   10000 non-null  int64 
 2   age                  9950 non-null   object
 3   gender               9950 non-null   object
 4   race                 9951 non-null   object
 5   driving_experience   9950 non-null   object
 6   education            9951 non-null   object
 7   income               10000 non-null  object
 8   credit_score         8978 non-null   object
 9   vehicle_ownership    9950 non-null   object
 10  vehicle_year         9950 non-null   object
 11  married              9952 non-null   object
 12  children             9950 non-null   object
 13  postal_code          9950 non-null   object
 14  annual_mileage       9000 non-null   object
 15  vehicle_type         9951 non-null   object
 16  speed

In [19]:
claims[claims.past_accidents.isna() & claims.outcome.isna()]

,unnamed: 0,id,age,gender,race,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome


In [20]:
claims.isnull().sum()

unnamed: 0                0
id                        0
age                      50
gender                   50
race                     49
driving_experience       50
education                49
income                    0
credit_score           1022
vehicle_ownership        50
vehicle_year             50
married                  48
children                 50
postal_code              50
annual_mileage         1000
vehicle_type             49
speeding_violations      50
duis                     50
past_accidents           50
outcome                  50
dtype: int64